In [1]:
from PIL import Image
from IPython.display import display
import glob,os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from skimage.transform import resize, downscale_local_mean
import scipy

In [2]:
mnhc_path = "données/MNHC/35_2020/" # "/Users/pmeurice/Documents/Projets/IGN-DSB/ML/data/inputs-ortho-256/"
ortho_path = "données/orthophotos/2006_OK/rvb/"

res_path = "/données/vignettes/"

In [3]:
orthophotos = glob.glob(ortho_path+"*.tif")

In [4]:
len(orthophotos)

369

In [5]:
orthophotos[4]

'données/orthophotos/2006_OK/rvb\\35-2006-0305-6780-LA93.tif'

In [7]:
mnhc = glob.glob(mnhc_path+"*.tif")

In [8]:
len(mnhc)

7198

In [9]:
mnhc[:5]

['données/MNHC/35_2020\\Diff_MNS_CORREL_1-0_LAMB93_20FD3525_305_6779.tif',
 'données/MNHC/35_2020\\Diff_MNS_CORREL_1-0_LAMB93_20FD3525_305_6780.tif',
 'données/MNHC/35_2020\\Diff_MNS_CORREL_1-0_LAMB93_20FD3525_305_6781.tif',
 'données/MNHC/35_2020\\Diff_MNS_CORREL_1-0_LAMB93_20FD3525_306_6779.tif',
 'données/MNHC/35_2020\\Diff_MNS_CORREL_1-0_LAMB93_20FD3525_306_6780.tif']

In [18]:
mnhc_path + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(orthophotos[4][-17:-9]).replace('-', '_') + ".tif"

'données/MNHC/35_2020/Diff_MNS_CORREL_1-0_LAMB93_20FD3525_305_6780.tif'

In [19]:
def std_convoluted(im, N):
    """
    Calculate the standard deviation using a convolution operation.

    Parameters:
    - im (numpy.ndarray): Input image as a NumPy array.
    - N (int): Radius of the convolution kernel.

    Returns:
    - numpy.ndarray: Standard deviation image.
    """
    # Calculate squared image and initialize kernel
    im2 = im**2
    ones = np.ones(im.shape)
    kernel = np.ones((2*N+1, 2*N+1))

    # Convolve the image and squared image with the kernel
    s = scipy.signal.convolve2d(im, kernel, mode="same")
    s2 = scipy.signal.convolve2d(im2, kernel, mode="same")
    ns = scipy.signal.convolve2d(ones, kernel, mode="same")

    # Calculate the standard deviation image
    return np.sqrt((s2 - s**2 / ns) / ns)

In [101]:
path_to_orthophoto = orthophotos[4]
IMAGE_SIZE = 256

# Extract position information from the orthophoto path
position = path_to_orthophoto[-17:-9] # CHANGEMENT orthophoto[-28:-19]
print(position, flush=True)

# Open the orthophoto and MNHC images
ortho = Image.open(path_to_orthophoto)
mnhc = Image.open(mnhc_path + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + path_to_orthophoto[-17:-9].replace('-', '_') + ".tif")
width, height = mnhc.size

# Resize the orthophoto to match the size of MNHC
ortho = ortho.resize(mnhc.size, resample=Image.Resampling.BICUBIC)
o = np.array(ortho)

# Calculate NDVI (Normalized Difference Vegetation Index)
ndvi = np.divide(o[:,:,0]-o[:,:,1], o[:,:,0]+o[:,:,1], where=(o[:,:,0]+o[:,:,1]) != 0)

npmnhc = np.array(mnhc)

# Calculate standard deviation of the MNHC using the provided function
std = std_convoluted(npmnhc, N=10)

# Create a mask based on specified conditions
mask = Image.fromarray(np.logical_and(
    np.logical_and(npmnhc > 3., # (npmnhc > 3. pour ne garder que les valeurs de hauteur strictement supérieures à 3 mètres afin de limiter les erreurs liées à l’imprécision du MNS
                   np.logical_or(npmnhc > 5., std > .5)), 
    ndvi > 0 # cad végétation
))

305-6780


In [47]:
(x - x % 2000, y - y % 2000, x - x % 2000 + 2000, y - y % 2000 + 2000)

(0, 0, 2000, 2000)

In [123]:
nonvalides = 0
i = 0
for x in range(0, width - IMAGE_SIZE, IMAGE_SIZE): #(0, width - IMAGE_SIZE, IMAGE_SIZE)
    for y in range(0, height - IMAGE_SIZE, IMAGE_SIZE): #(0, height - IMAGE_SIZE, IMAGE_SIZE)

        # i+=1
        # y = 0
        # Test if MNHC is valid
        xw = x + IMAGE_SIZE - 1
        yh = y + IMAGE_SIZE - 1
        # print(i)
        # print((x, y, xw, yh))
        # print((x - x % 2000, y - y % 2000, x - x % 2000 + 2000, y - y % 2000 + 2000))
        # print((xw - xw % 2000, yh - yh % 2000, xw - xw % 2000 + 2000, yh - yh % 2000 + 2000))
        # print((x - x % 2000, yh - yh % 2000, x - x % 2000 + 2000, yh - yh % 2000 + 2000))
        # print((xw - xw % 2000, y - y % 2000, xw - xw % 2000 + 2000, y - y % 2000 + 2000))
        # plt.imshow(np.array(mnhc.crop((x - x % 2000, y - y % 2000, x - x % 2000 + 2000, y - y % 2000 + 2000))))
        # print('')
        
        if np.array(mnhc.crop((x - x % 2000, y - y % 2000, x - x % 2000 + 2000, y - y % 2000 + 2000))).sum() == 0 or \
           np.array(mnhc.crop((xw - xw % 2000, y - y % 2000, xw - xw % 2000 + 2000, y - y % 2000 + 2000))).sum() == 0 or \
           np.array(mnhc.crop((x - x % 2000, yh - yh % 2000, x - x % 2000 + 2000, yh - yh % 2000 + 2000))).sum() == 0 or \
           np.array(mnhc.crop((xw - xw % 2000, yh - yh % 2000, xw - xw % 2000 + 2000, yh - yh % 2000 + 2000))).sum() == 0:
            nonvalides = nonvalides + 1
            
            continue

        # Create a filename for the vignette
        vname = "35-2020-"+position+"-"+str(x).zfill(4)+"-"+str(y).zfill(4)+"-050M-"

        # Save the orthophoto and mask as vignettes
        ortho.crop((x, y, x + IMAGE_SIZE, y + IMAGE_SIZE)).save("vignettes/inputs-ortho-256/"+vname+"ortho.jpg")
        v_mask = mask.crop((x, y, x + IMAGE_SIZE, y + IMAGE_SIZE))
        v_mask.save("vignettes/annotations-256/"+vname+"mask.png")

print(position+" : "+str(nonvalides)+" non valides", flush=True)

305-6780 : 0 non valides


In [106]:
x, y, x + IMAGE_SIZE, y + IMAGE_SIZE

(1792, 1792, 2048, 2048)

In [67]:
print(f"""
x: {x}
y: {y}
xw: {xw}
yh: {yh}
""")


x: 1536
y: 0
xw: 1791
yh: 255

